In [76]:
import numpy as np

### Input data

In [77]:
start_point = [5.2, 5.2]
lambda_step = 1
hesse_matrix = [[4, 1], [1, 12]]


def target_function(x1, x2):
    return 2*(x1 - 1)**2 + x1*x2 + 6*x2**2


def target_function_dx1(x1, x2):
    return 4*x1 + x2 - 4


def target_function_dx2(x1, x2):
    return x1 + 12*x2


### Helper functions

In [78]:
def vector_sum(v1, v2):
    res = v1.copy()
    for i in range(len(res)):
        res[i] += v2[i]
    return res


def multiply_vector(v, number):
    res = v.copy()
    for i in range(len(v)):
        res[i] *= number
    return res


def print_points(points):
    for i in range(len(points)):
        point = "("
        for j in range(len(points[i])):
            point += str(points[i][j])
            if j != len(points[i]) - 1:
                point += " ; "
        point += ")"
        print("X", i + 1, " = ", point)

### Shortest descent method

In [79]:
def calculate_s(point, gradient):
    gradient_values = []
    for i in range(len(gradient)):
        gradient_values.append(gradient[i](*point))

    gradient_module = np.linalg.norm(np.array(gradient_values))
    vector_s = []

    for i in range(len(gradient_values)):
        vector_s.append(-1*gradient_values[i]/gradient_module)

    return vector_s, gradient_values


def calculate_optimal_step(s_vector, gradient_values, hesse):
    s = np.array(s_vector)
    g = np.array(gradient_values)
    h = np.array(hesse)

    a = g.transpose().dot(s)
    b = s.transpose().dot(h).dot(s)

    return -1 * a / b


def shortest_descent(x0, analytical_gradient, iterations=3, default_step=1, hesse=[]):
    point = x0
    res = []
    for i in range(iterations):
        vector_s, gradient_values = calculate_s(point, analytical_gradient)
        if default_step:
            step = default_step
        else:
            step = calculate_optimal_step(vector_s, gradient_values, hesse)
        point = vector_sum(point, multiply_vector(vector_s, step))
        res.append(point)
    return res

### Shortest descent method without optimal step

In [80]:
shortest_descent_points = shortest_descent(start_point, [target_function_dx1, target_function_dx2], 3, lambda_step)
print_points(shortest_descent_points)

X 1  =  (4.890532266671866 ; 4.249090055773552)
X 2  =  (4.556377564731074 ; 3.306571814480122)
X 3  =  (4.187923979668806 ; 2.376925669081367)


### Shortest descent method with optimal step

In [81]:
shortest_descent_points = shortest_descent(start_point, [target_function_dx1, target_function_dx2], 3, None, hesse_matrix)
print_points(shortest_descent_points)

X 1  =  (3.3391241008831836 ; -0.5179641263771284)
X 2  =  (1.223433174452481 ; 0.17057434080446132)
X 3  =  (1.1334084782062126 ; -0.10604699857043537)


### Partan's method

In [82]:
def shortest_descent_partan(x0, analytical_gradient, hesse=[], iterations=3):
    point = x0
    res = []
    for i in range(iterations):
        vector_s, gradient_values = calculate_s(point, analytical_gradient)
        if i >= 2:
            vector_s = np.subtract(np.array(res[len(res) - 1]), np.array(x0))
        step = calculate_optimal_step(vector_s, gradient_values, hesse)
        point = vector_sum(point, multiply_vector(vector_s, step))
        res.append(point)
    return res


shortest_descent_points = shortest_descent_partan(start_point, [target_function_dx1, target_function_dx2], hesse_matrix, 3)
print_points(shortest_descent_points)

X 1  =  (3.3391241008831836 ; -0.5179641263771284)
X 2  =  (1.223433174452481 ; 0.17057434080446132)
X 3  =  (1.021276595744681 ; -0.08510638297872347)


### Newton's method

In [83]:
def newton_method(x0, analytical_gradient, hesse, iterations = 3):
    point = x0
    res = [point]
    inv_hesse = np.linalg.inv(np.array(hesse))
    for i in range(iterations):
        gradient_values = []
        for j in range(len(analytical_gradient)):
            gradient_values.append(analytical_gradient[j](*point))
        gradient_values = np.array(gradient_values)
        point = np.subtract(np.array(point), inv_hesse.dot(gradient_values))
        res.append(point)
    return res


newton_points = newton_method(start_point, [target_function_dx1, target_function_dx2], hesse_matrix, 3)
print_points(newton_points)

X 1  =  (5.2 ; 5.2)
X 2  =  (1.0212765957446814 ; -0.08510638297872308)
X 3  =  (1.0212765957446808 ; -0.0851063829787234)
X 4  =  (1.021276595744681 ; -0.08510638297872342)
